<a href="https://colab.research.google.com/github/Fermu25/Cursos/blob/main/Calibracion_sensores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

class SensorModel:
    def __init__(self):
        # Modelo inicial (se recalculará cuando haya 2 puntos nuevos)
        self.a = 0.00139027
        self.b = -10578.1
        self.err_a = 1.365e-6
        self.err_b = 15.44
        self.historial = []

    def estimar_presion(self, bits):
        presion = self.a * bits + self.b
        return presion

    def estimar_incertidumbre(self, bits):
        return np.sqrt((bits * self.err_a) ** 2 + self.err_b ** 2)

    def evaluar(self, bits, presion_real):
        estimada = self.estimar_presion(bits)
        incertidumbre = self.estimar_incertidumbre(bits)
        error_abs = presion_real - estimada
        error_pct = 100 * error_abs / presion_real if presion_real != 0 else float('inf')

        print(f"\nResultado de evaluación:")
        print(f"Bits leídos:   {bits}")
        print(f"Presión real:      {presion_real:.2f} Pa")
        print(f"Presión estimada:  {estimada:.2f} Pa ± {incertidumbre:.2f} Pa")
        print(f"Error:             {error_abs:.2f} Pa ({error_pct:.2f}%)")

        self.imprimir_modelo_actual()
        return estimada, incertidumbre

    def agregar_dato(self, bits, presion_real):
        self.historial.append((bits, presion_real))
        if len(self.historial) >= 2:
            self.recalcular_modelo()
        else:
            print("\n Se necesita al menos 1 punto más para ajustar la ecuación.")

    def recalcular_modelo(self):
        X, Y = zip(*self.historial)
        X = np.array(X)
        Y = np.array(Y)
        A = np.vstack([X, np.ones_like(X)]).T
        resultado = np.linalg.lstsq(A, Y, rcond=None)
        coef, residuals, _, _ = resultado

        self.a, self.b = coef

        # Cálculo de errores estándar
        y_est = A @ coef
        resid = Y - y_est
        sse = np.sum(resid ** 2)
        dof = len(X) - 2
        var_a = sse / (dof * np.sum((X - np.mean(X)) ** 2)) if dof > 0 else 0
        var_b = var_a * np.mean(X ** 2) if dof > 0 else 0
        self.err_a = np.sqrt(var_a)
        self.err_b = np.sqrt(var_b)

        print(f"\n✅ Modelo recalculado con {len(X)} puntos.")

    def imprimir_modelo_actual(self):
        print(f"\n📐 Modelo actual:")
        print(f"   P = {self.a:.8f} * bits + {self.b:.4f}")
        print(f"   ±{self.err_a:.2e} (pendiente), ±{self.err_b:.2f} (intercepto)\n")

# ========================
print("Calibración de sensor de presión\n")
modelo = SensorModel()

print("Introduce 2 puntos (bits y presión real en Pa)")
print("Presiona Ctrl+C para salir.\n")

while True:
    try:
        bits = int(input("Bits leídos en Arduino: "))
        presion_real = float(input("Presión real medida (Pa): "))

        modelo.agregar_dato(bits, presion_real)
        modelo.evaluar(bits, presion_real)


SyntaxError: incomplete input (<ipython-input-3-05487956a7b1>, line 81)

In [ ]:
import numpy as np
class SensorModel:
    def __init__(self):
        self.a = 0.00139027
        self.b = -10578.1
        self.err_a = 1.365e-6
        self.err_b = 15.44
        self.historial = []

    def estimar_presion(self, bits):
        presion = self.a * bits + self.b
        return presion

    def estimar_incertidumbre(self, bits):
        return np.sqrt((bits * self.err_a) ** 2 + self.err_b ** 2)

    def evaluar(self, bits, presion_real):
        estimada = self.estimar_presion(bits)
        incertidumbre = self.estimar_incertidumbre(bits)
        error_abs = presion_real - estimada
        error_pct = 100 * error_abs / presion_real if presion_real != 0 else float('inf')

        print(f"\nResultado de evaluación:")
        print(f"Bits leídos:         {bits}")
        print(f"Presión real:        {presion_real:.2f} Pa")
        print(f"Presión estimada:    {estimada:.2f} Pa ± {incertidumbre:.2f} Pa")
        print(f"Error absoluto:      {error_abs:.2f} Pa")
        print(f"Error porcentual:    {error_pct:.2f}%")

    def agregar_dato(self, bits, presion_real):
        self.historial.append((bits, presion_real))
        if len(self.historial) >= 2:
            self.recalcular_modelo()

    def recalcular_modelo(self):
        X, Y = zip(*self.historial[-2:])  # Solo usa los últimos 2 puntos
        X = np.array(X)
        Y = np.array(Y)
        A = np.vstack([X, np.ones_like(X)]).T
        resultado = np.linalg.lstsq(A, Y, rcond=None)
        coef, residuals, _, _ = resultado

        self.a, self.b = coef

        # Cálculo de errores estándar
        y_est = A @ coef
        resid = Y - y_est
        sse = np.sum(resid ** 2)
        dof = len(X) - 2
        var_a = sse / (dof * np.sum((X - np.mean(X)) ** 2)) if dof > 0 else 0
        var_b = var_a * np.mean(X ** 2) if dof > 0 else 0
        self.err_a = np.sqrt(var_a)
        self.err_b = np.sqrt(var_b)

        print(f"\n Modelo recalculado con los últimos 2 puntos.")
        self.imprimir_modelo_actual()

    def imprimir_modelo_actual(self):
        print(f"\n  Modelo actual:")
        print(f"   P = {self.a:.8f} * bits + {self.b:.4f}")
        print(f"   ±{self.err_a:.2e} (pendiente), ±{self.err_b:.2f} (intercepto)\n")


# ========================
# Ejecución interactiva
# ========================

print("=== Calibración de sensor de presión ===\n")

modelo = SensorModel()

continuar = "s"

while continuar.lower() == "s":
    print("\nIntroduce 2 puntos nuevos (bits y presión real en Pa):")

    for i in range(2):
        print(f"\n🔹 Punto {i+1}:")
        bits = int(input("   Bits leídos en Arduino: "))
        presion_real = float(input("   Presión real medida (Pa): "))
        modelo.agregar_dato(bits, presion_real)
        modelo.evaluar(bits, presion_real)

    continuar = input("\n¿Quieres ingresar otros 2 puntos? (s/n): ")

print("\n Proceso finalizado.")


=== Calibración de sensor de presión ===


Introduce 2 puntos nuevos (bits y presión real en Pa):

🔹 Punto 1:
   Bits leídos en Arduino: 23453
   Presión real medida (Pa): 34

Resultado de evaluación:
Bits leídos:         23453
Presión real:        34.00 Pa
Presión estimada:    -10545.49 Pa ± 15.44 Pa
Error absoluto:      10579.49 Pa
Error porcentual:    31116.16%

🔹 Punto 2:
   Bits leídos en Arduino: 344563
   Presión real medida (Pa): 35

 Modelo recalculado con los últimos 2 puntos.

  Modelo actual:
   P = 0.00000311 * bits + 33.9270
   ±0.00e+00 (pendiente), ±0.00 (intercepto)


Resultado de evaluación:
Bits leídos:         344563
Presión real:        35.00 Pa
Presión estimada:    35.00 Pa ± 0.00 Pa
Error absoluto:      -0.00 Pa
Error porcentual:    -0.00%

¿Quieres ingresar otros 2 puntos? (s/n): n

 Proceso finalizado.
